In [ ]:
!pip install flwr

In [ ]:
import flwr as fl
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Create synthetic fraud dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to train logistic regression model locally
def train_local_model(X_train, y_train):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    return model

# Function to evaluate logistic regression model
def evaluate_model(model, X_test, y_test):
    accuracy = model.score(X_test, y_test)
    return accuracy

# Function to create client
def create_client(X_train, y_train, X_test, y_test):
    class Client(fl.client.NumPyClient):
        def get_parameters(self):
            return np.concatenate([model.coef_.ravel(), model.intercept_])

        def set_parameters(self, parameters):
            model.coef_ = parameters[:-1].reshape(1, -1)
            model.intercept_ = parameters[-1]

        def fit(self, parameters, config):
            self.set_parameters(parameters)
            local_model = train_local_model(X_train, y_train)
            return self.get_parameters(), len(X_train), {}

        def evaluate(self, parameters, config):
            self.set_parameters(parameters)
            return evaluate_model(model, X_test, y_test), len(X_test), {}

    return Client()

# Define the number of clients
num_clients = 5

# Split data and create clients
clients = []
for _ in range(num_clients):
    X_train_local, X_test_local, y_train_local, y_test_local = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    client = create_client(X_train_local, y_train_local, X_test_local, y_test_local)
    clients.append(client)

# Start Flower server on a different port
server = fl.server.start_server(server_address = "0.0.0.0:8081", config=fl.server.ServerConfig(num_rounds=3))  # Adjust num_rounds as needed

# Connect clients to the Flower server
server.client_manager.add_clients(clients)


INFO :      Starting Flower server, config: num_rounds=3, no round_timeout
INFO:flwr:Starting Flower server, config: num_rounds=3, no round_timeout
INFO :      Flower ECE: gRPC server running (3 rounds), SSL is disabled
INFO:flwr:Flower ECE: gRPC server running (3 rounds), SSL is disabled
INFO :      [INIT]
INFO:flwr:[INIT]
INFO :      Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client


KeyboardInterrupt: 